# У вас есть данные АБ-теста алгоритма ранжирования в новостях

Данные лежат в [файле](https://github.com/pileyan/HSE_Statistics_1/blob/main/lect%2010%20CUPED/news%20test.csv)

Каждая строка - статистика о том, какой набор действий совершил за этот день пользователь (строка = пользователь+день)

**d** - День

**exp_group** -  id экспериментальной группы

**bot_flg** - 1 - мы считаем пользователя ботом, 0 - считаем живым пользователем

**user_info_yandex_uid** - уникальный id пользователя

**total_clicks_** - сколько кликов по поисковой выдаче сделал пользователь за день

**time_to_first_click_** - Если пользователь хоть раз кликнул по поисковой выдаче по запросу, то для каждого запроса высчитывается "какое время прошло от показа выдачи до 1 клика". Если человек сделал несколько запросов, по каждому из них считается время до первого клика, а затем суммируется 

**first_click_pos_** - Если пользователь хоть раз кликнул по поисковой выдаче по запросу, то для каждого запроса высчитывается "В какую именно позицию пользователь кликнул первой". Если человек сделал несколько запросов, по каждому из тех, в которых был клик, считается позиция первого клика, а затем суммируется 

**sum_pos** - Выдача в новостях устроена "блоками". Позиции в каждом из блоков можно пронумеровать (каждый новый блок начинается с 1). Если человек совершил клик в некоторую позицию, считается ее номер внутри блока и суммируется с позициями других его кликов

**end2end_sum_pos** - По выдаче можно ввести "сквозную нумерацию", то есть не "внутри каждого блока начинаем с 1", а просто нумеруем от начала и до конца. Далее берем все клики пользователя, считаем "сквозную позицию кликнутых документов" и суммируем 

**queries** - количество запросов за день 

**empty_results** - количество запросов , вернувших пустую выдачу

# Задание 1

Проверьте с помощью подходящего теста (и поясните, почему этот тест хорошо подходит для этой метрики) статистическую значимость отличия в следующем списке метрик:


1) Среднее количество кликов на пользователя за весь период эксперимента

2) Средняя позиция клика (сумма позиций/кол-во кликов)

3) Средняя сквозная позиция клика (сумма сквозных позиций/кол-во кликов)

4) Доля пустых результатов выдачи


Также очень рекомендуется провести EDA и посмотреть, нет ли в данных чего-то интересного) Хороший EDA поможет вам лучше разобраться, что , как и с чем взаимосвязано.

# Задание 2

Постройте графики метрик из пунктов 1-4 по дням, чтобы посмотреть, виден ли эффект, который вы измерили с помощью теста, невооруженным глазом. А также - является ли он стабильным во времени.

# Задание 3

Проанализируйте результаты пунктов 1-4 , убрав из групп ботов. Изменились ли выводы?


In [1]:
import duckdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('../lect 10 CUPED/news test.csv')

In [6]:
df = df.drop(['is_biterika', 'is_yandex', 'is_sber', 'is_rntv',
       'is_vtb', 'is_telegram_bot', 'is_yandex_bot', 'is_medialogia',
       'is_selectel', 'is_interfax', 'is_RT'], axis = 1)

df= df.drop(['user_info_user_agent', 'kommersant',
       'kommersant_shows', 'rbc', 'rbc_shows', 'ria', 'ria_shows', 'lenta',
       'lenta_shows', 'tass', 'tass_shows', 'iz', 'iz_shows', 'interfax',
       'interfax_shows'], axis = 1)

df['dt'] = pd.to_datetime(df.d, format = '%d/%m/%y')

In [7]:
df

,d,exp_group,bot_flg,user_info_yandex_uid,total_clicks_,time_to_first_click_,first_click_pos_,sum_pos,end2end_sum_pos,queries,empty_results,dt
0,26/04/23,control,0,1865850381682515486,0,0,0,0,0,1,1,2023-04-26
1,28/04/23,хронология,0,8022259061682679060,0,0,0,0,0,1,0,2023-04-28
2,28/04/23,control,1,8516704241675260594,0,0,0,0,0,278,203,2023-04-28
3,01/05/23,control,0,6430450711682974597,0,0,0,0,0,1,0,2023-05-01
4,27/04/23,хронология,0,5047628321682545957,0,0,0,0,0,1,0,2023-04-27
...,...,...,...,...,...,...,...,...,...,...,...,...
251688,28/04/23,control,0,4935607761674553118,0,0,0,0,0,2,0,2023-04-28
251689,26/04/23,хронология,0,4832233251682471511,0,0,0,0,0,1,0,2023-04-26
251690,29/04/23,control,0,5520044421682742770,0,0,0,0,0,1,0,2023-04-29
251691,29/04/23,control,0,9995464511682731826,0,0,0,0,0,1,0,2023-04-29


In [13]:

q = '''
select exp_group, 
    count_if(clicks=0) as zero_clicks,
    sum(1) as total,
    count_if(clicks=0)/sum(1) as zero_clicks_ratio
from (
SELECT user_info_yandex_uid, exp_group, sum(total_clicks_) as clicks
FROM df
WHERE bot_flg = 0
GROUP BY user_info_yandex_uid, exp_group
)
group by exp_group
'''

duckdb.query(q).to_df()

,exp_group,zero_clicks,total,zero_clicks_ratio
0,хронология,109855.0,113453.0,0.968286
1,control,109822.0,113382.0,0.968602


# Показатели, которые касаются кликов, нужно считать ТОЛЬКО по тем, кто кликнул хоть раз. 

## А похоже ли у низ количество запросов

In [19]:

q = '''
select exp_group, 
    sum(queries_empty_results) as empty_results,
    sum(queries) as queries,
    sum(queries_empty_results)/sum(queries) as empty_results_ratio
from (
SELECT user_info_yandex_uid, exp_group, 
    sum(queries) as queries,
    sum(empty_results) as queries_empty_results
FROM df
where bot_flg = 0
GROUP BY user_info_yandex_uid, exp_group
)
group by exp_group
'''

duckdb.query(q).to_df()

,exp_group,empty_results,queries,empty_results_ratio
0,control,39357.0,179146.0,0.219692
1,хронология,13375.0,175346.0,0.076278


## По сколько запросов делают боты и не-боты

In [24]:

q = '''
select exp_group, 
    bot_flg,
    avg(queries) as avg_queries,
    quantile_disc(queries, [0.25, 0.5, 0.75, 0.99]) as quantiles_queries,
    max(queries) as max_queries
from (
SELECT user_info_yandex_uid, exp_group, 
    bot_flg,
    sum(queries) as queries
FROM df
GROUP BY user_info_yandex_uid, exp_group, bot_flg
)
group by exp_group, bot_flg
'''

duckdb.query(q).to_df()

,exp_group,bot_flg,avg_queries,quantiles_queries,max_queries
0,хронология,1,34.329277,"[1.0, 1.0, 1.0, 1999.0]",2541.0
1,control,0,1.580022,"[1.0, 1.0, 1.0, 7.0]",4428.0
2,control,1,41.174768,"[1.0, 1.0, 1.0, 2019.0]",2521.0
3,хронология,0,1.545539,"[1.0, 1.0, 1.0, 9.0]",1994.0


In [25]:
q = '''
select groups, sum(1)
from (
SELECT user_info_yandex_uid, count(distinct exp_group) as groups
FROM df
GROUP BY user_info_yandex_uid
)
group by groups
'''

duckdb.query(q).to_df()

,groups,sum(1)
0,1,237951.0
1,2,20.0


## Посмотрите, как ведут себя те, кто попал в обе группы